# Expectation Maximization 

Expectation Maximization (EM) is an algorithm you can use to estimate the maximum likelihood for the value of latent (hidden) parameters. Suppose, for example, that your friend possesses a bag of five unfair coins, but you don't know the "weights" of the coins--i.e., each coin's probability of coming up heads. Your friend offers to shake the bag, pick a coin out of the bag, flip it 100 times, then put it back in the bag. He goes through this cycle this 100 times. The visible parameters are the 100 sessions of heads or tails; the latent parameters are the "weights" of the five coins. How can you estimate the latent parameters? EM provides a useful method.

### Simulation scenario

In a simulation deviously devised by Professor David Crandall, a candy company has 5 different probabilities of inserting cherry candies into bags of cherry and lime candies. You, the student, do not know what these probabilities are. However, you are allowed to enter a dark truck and randomly select 100 bags of candy. You may then draw 100 candies from each bag. Your goal is to estimate the 5 cherry probabilities--the latent parameters--based on your observations. You must also provide a maximum likelihood estimate of which of the 5 probability distributions was used by the manufacturer for each of the 100 bags.

*Note: The setup code was written by Professor Crandall. I did insert one optimization by converting a list to a numpy array, though.*

In [3]:
import random
import numpy as np
import math

#####
# You shouldn't have to modify this part

# These are the *actual* values of C0 ... C4 we're trying to estimate.
# Shh... They're a secret! :) This is what we're trying to estimate.
bagtype_count = 5
actual_cs = (0.2, 0.3, 0.7, 0.9, 1.0)

# Now sample 100 bags
bag_count = 100
actual_bagtypes = [ random.randrange(0, bagtype_count) for _ in range(0, bag_count) ]

# Now sample 100 candies from each bag, to produce a list-of-lists
# The result is a list of bag_count lists. Each list has candy_count members that are either "X" or "C", 
# drawn randomly according to a distribution
candy_count = 100
observations = [ [ ("L", "C")[x] for x in tuple(np.random.binomial( 1, actual_cs[ bagtype ], candy_count ) ) ] 
                 for bagtype in actual_bagtypes ] 

# This list will hold your estimated C0 ... C4 values, and your estimated
# bagtype for each bag.
estimated_cs = [0] * bagtype_count
estimated_bagtypes = np.array([0] * bag_count) # cfalter: changed from list to ndarray


### The Algorithm
The algorithm iteratively performs two steps until the estimates of the latent parameters converge:
**1. Estimation (E)**: Given a set of labels for the latent parameters, estimate (assign) the label corresponding to the most likely parameter. In our scenario, the labels correspond to probabilities used by the manufacturer when packaging each of the bags.
**2. Maximization (M)**: For each label, estimate the parameter value that maximizes the likelihood of its corresponding observations. In our scenario, the parameter values is a probability of drawing cherry candies, and we calculate it as the mean of the distributions assigned to a label.

#### Initialization: Chicken or egg? A random answer
You need a set of parameter values to perform the Estimation step that assigns labels; but you need labeled distributions to perform the Maximization step. Since each step depends on the output of the other, how can we start the algorithm? The answer is simple: you initialize by assigning random parameter values. 

#### How likely is the result? 
The likelihood of each possible label for each bag is calculated in the Estimation step: The maximum likelihood indicates the best label. Dividing the highest parameter likelihood by the sum of all parameter likelihoods yields the likelihood that the label represents the ground truth. We can multiply these likelihoods together to find the likelihood that a given set of parameters is the ground truth. This cumulative likelihood increases with each iteration of the algorithm and reaches a plateau when the algorithm converges.

Because this likelihood can be quite low (as low as 20%) and we have 100 observations, it is better to sum the log of the likelihoods as opposed to multiplying the likelihoods. This relies on the logarithm product rule: `log(ab) = log(a) + log(b)`.

In [4]:
import scipy.stats as ss

C_counts = np.array([np.sum(np.array(observations[i]) == 'C') for i in range(bag_count)])

def em(counts):
    # E-M parameters
    max_iters = 100 # if E-M algorithm hasn't converged after this many iterations, just stop
    bagtypes = np.array([0] * bag_count)
    
    # randomly initialize estimates of c (probability of drawing cherry candy from a bag)
    cs = np.random.uniform(0.001, 1.0, bagtype_count)
    old = np.copy(cs)
    
    # run EM
    for i in range(max_iters):
        loglikelihood = 0.0
        likelihoodFns = [ss.binom(candy_count, c) for c in cs]
        
        # E-step
        for bag in range(bag_count):
            C = counts[bag]
            likelihoods = np.array([likelihoodFns[j].pmf(C) for j in range(bagtype_count)])
            most_likely = np.argmax(likelihoods)
            bagtypes[bag] = most_likely
            loglikelihood_b = math.log(likelihoods[most_likely]/np.sum(likelihoods))
            loglikelihood += loglikelihood_b
        
        # M-step
        for j in range(bagtype_count):
            if np.sum(bagtypes == j) == 0:
                # a bad initialization resulted in an estimated_c that was never most likely
                # for any of the bags. Do a random restart for that estimated_c
                cs[j] = random.uniform(0.001, 1.0)
            else:
                cs[j] = np.mean(counts[bagtypes == j]) / float(candy_count)
            
        if np.all(old == cs):
            break
        
        old = np.copy(cs)
        
    return loglikelihood, cs, bagtypes


### The Problem of Local Maxima
The EM algorithm often converges to a very good approximation of the latent parameters. Unfortunately, this happy result is, like most happy results in life, not guaranteed. A quirky random initialization of the latent parameters can cause the algorithm to converge to a local maximum likelihood that is not the global maximum likelihood. For example, a fairly frequent parameter estimation in the cherry candy scenario was the set {0.3, 0.3, 0.66, 0.74, 0.95}--compared to the ground truth of {0.2, 0.3, 0.7, 0.9, 1.0}. Therefore I ran the EM algorithm 25 times and selected the one output of the 25 that maximized the likelihood of the observations. If I recall correctly, the probability that the algorithm would reach a local rather than a global maximum in this scenario was about 1/4. Thus the probability that the global maximum will not be found by running the algorithm 25 times is (1/4)<sup>25</sup>, or about 1 in 1,125,899,910,000,000. In a word, infinitesimal.

In [5]:
num_Ems = 25
results = []
for i in range(num_Ems):
    likelihood, cs, bagtypes = em(C_counts)
    results.append((likelihood, cs, bagtypes))

# sort on likelihood and select the winner    
winner = sorted(results, key = lambda x: x[0], reverse = True)[0]
estimated_cs = winner[1]
estimated_bagtypes = winner[2]

### Print and You're Done
The remainder of the code was provided by Professor Crandall so he and the TAs could grade the assignment.

In [7]:
sorted_cs = sorted((e,i) for i,e in enumerate(estimated_cs))
estimated_cs = [ v[0] for v in sorted_cs ]
index_remapper = [ 0 ] * bagtype_count
for i in range(0, bagtype_count):
    index_remapper[ sorted_cs[i][1] ] = i
estimated_bagtypes = [ index_remapper[bagtype] for bagtype in estimated_bagtypes ]

print ("Actual C's:         ", actual_cs)
print ("Estimated C's:      ", estimated_cs)
print ("Actual bagtypes:    ", actual_bagtypes)
print ("Estimated bagtypes: ", estimated_bagtypes)
print ("Correctly estimated bags: ", sum( [ actual_bagtypes[i] == estimated_bagtypes[i] for i in range(0, len(estimated_bagtypes) ) ] ))


Actual C's:          (0.2, 0.3, 0.7, 0.9, 1.0)
Estimated C's:       [0.16250000000000001, 0.27740740740740738, 0.71400000000000008, 0.89631578947368429, 1.0]
Actual bagtypes:     [2, 3, 1, 3, 1, 1, 0, 3, 0, 3, 4, 4, 1, 0, 3, 1, 3, 0, 3, 0, 1, 3, 2, 2, 4, 4, 1, 4, 1, 1, 3, 4, 1, 1, 4, 0, 2, 2, 2, 4, 4, 1, 4, 3, 4, 2, 0, 4, 2, 0, 0, 3, 1, 4, 0, 0, 0, 2, 3, 1, 1, 2, 3, 1, 4, 0, 3, 2, 2, 3, 3, 4, 2, 1, 2, 4, 0, 4, 4, 4, 0, 3, 2, 1, 2, 1, 0, 2, 3, 4, 3, 2, 2, 2, 1, 0, 4, 0, 0, 4]
Estimated bagtypes:  [2, 3, 1, 3, 1, 1, 0, 3, 1, 3, 4, 4, 1, 0, 3, 1, 3, 0, 3, 1, 1, 3, 2, 2, 4, 4, 1, 4, 1, 1, 3, 4, 1, 1, 4, 0, 2, 2, 2, 4, 4, 1, 4, 3, 4, 2, 0, 4, 2, 0, 1, 3, 1, 4, 0, 0, 1, 2, 3, 1, 1, 2, 3, 1, 4, 1, 3, 2, 2, 3, 3, 4, 2, 1, 2, 4, 0, 4, 4, 4, 1, 3, 2, 1, 2, 1, 0, 2, 3, 4, 3, 2, 2, 2, 1, 1, 4, 0, 0, 4]
Correctly estimated bags:  93
